## Introduction

In recent time, large, pre-trained, language models had shown high potential is several tasks. Such as Question Answering, Sentiment Analysis, Abstractive Summary etc. Some of the important models include [ELMo (Peters
et al., 2018)](https://arxiv.org/abs/1802.05365), [GPT (Radford et al., 2018)](https://arxiv.org/abs/2005.14165), [BERT (Devlin et al., 2018)](https://arxiv.org/pdf/1810.04805.pdf), [XLNet (Yang et al., 2019)](https://arxiv.org/pdf/1906.08237.pdf), and [RoBERTa (Liu et al., 2019)](https://arxiv.org/pdf/1907.11692.pdf).

They all follow the base architecture proposed by Vaswani et. al. in their Seminal Paper: [Attention is All You Need (Vaswani et al., 2017)](https://arxiv.org/abs/1706.03762)

Following the The naturalness hypothesis of source code, proposed by Allamanis et. al. it is thus preferable to try to treat large code corpora in the similar fashion and exploit their Statistical Properties. 

In this colab, we will see an end to end pipeline, using [Hugging Face Transformers Library](https://huggingface.co/transformers/), [Microsoft's Open Source Large Scale code model CodeBERT](https://huggingface.co/microsoft/codebert-base), and [Codist tree-hugger](https://github.com/autosoft-dev/tree-hugger) how to use similar technology to a very challenging problem called Code Summarization

## Background

![transformer](https://lilianweng.github.io/lil-log/assets/images/transformer.png)

<h3 align="center">The full model architecture of the transformer. (Image source: Fig 1 & 2 in Vaswani, et al., 2017.)</h3>


We won't go in detail of the transformer architecure. Because that is not really in the scope of this tutorial and there are plenty of very good resources that do justice to it. Please check out [here](https://jalammar.github.io/illustrated-transformer/), [here](http://nlp.seas.harvard.edu/2018/04/03/attention.html), and [here](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html). 

For this particular tutorial we are using Microsoft CodeBERT as the baseline model and this model has been proposed in a [paper](https://arxiv.org/pdf/2002.08155.pdf) in 2020 by Zhangyin Feng et. al. It is [freely available](https://huggingface.co/microsoft/codebert-base) via Hugging Face model repository. 

We at [Codist](https://codist-ai.com/) has released a new model and a tool called `docly` that does exactly same work (and a lot more!). We found our model slightly outperforming MS CodeBERT in some cases. If you are interested in it, please go to the website and sign up for a beta.

MS CodeBERT has been trained on a hybrid objective function where the model was simultaneously predicting both masked tokens and replaced tokens. So the final loss of the model can be expressed by

$$
min_\theta = L_{MLM}(\theta) + L_{RTD}(\theta)
$$


## Let's start the coding

We first download two companion files where we have some useful function and also the main model architecture code

In [ ]:
!wget https://raw.githubusercontent.com/autosoft-dev/ml-on-code/main/assets/model.py
!wget https://raw.githubusercontent.com/autosoft-dev/ml-on-code/main/assets/utils.py

--2022-03-29 22:09:57--  https://raw.githubusercontent.com/autosoft-dev/ml-on-code/main/assets/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9935 (9.7K) [text/plain]
Saving to: ‘model.py’

model.py            100%[===================>]   9.70K  --.-KB/s    in 0s      

2022-03-29 22:09:57 (60.6 MB/s) - ‘model.py’ saved [9935/9935]

--2022-03-29 22:09:57--  https://raw.githubusercontent.com/autosoft-dev/ml-on-code/main/assets/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2602 (2.5K) [text/plain]
Saving to: ‘ut

### Let's install tree-hugger and transformers

In [ ]:
!pip install transformers
!pip install -U tree-hugger PyYAML

     |████████████████████████████████| 3.8 MB 4.4 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 6.5 MB 37.6 MB/s 
     |████████████████████████████████| 895 kB 54.5 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 116 kB 6.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.5 MB 40.2 MB/s 
  Created wheel for tree-sitter: filename=tree_sitter-0.20.0-cp37-cp37m-linux_x86_64.whl size=304865 sha256=f2f36097eedfe5c6476006a3317a8b40fa89d2a83191666c2485ff9ff70c09f9
  Stored in directory: /root/.cache/pip/wheels/73/70/97/43e08c95ce68348303fd3f71adfa96193db8d9cf60a14163e0
Successfully built tree-sitter


### And use this command to build the necessary processing libary (tree-hugger related)

In [ ]:
!create_libs -c python

2022-03-29 22:10:57,383 INFO:Cloneing python repo from tree-sitter collections
2022-03-29 22:11:16,690 INFO:Creating the library my-languages.so at /content
2022-03-29 22:11:17,787 INFO:Finished creating library!


**Let's import all necessary modules**

In [ ]:
import os
import json
import torch
import torch.nn as nn
from model import Seq2Seq
from utils import Example, convert_examples_to_features
from transformers import RobertaConfig, RobertaModel, RobertaTokenizer
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

Now that we have everything, let's download the fine-tuned model. Codist has fine tuned this model for your testing purpose 😃

In [ ]:
!wget https://code-summary.s3.amazonaws.com/pytorch_model.bin

--2022-03-29 23:09:18--  https://code-summary.s3.amazonaws.com/pytorch_model.bin
Resolving code-summary.s3.amazonaws.com (code-summary.s3.amazonaws.com)... 52.217.134.49
Connecting to code-summary.s3.amazonaws.com (code-summary.s3.amazonaws.com)|52.217.134.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706871064 (674M) [application/macbinary]
Saving to: ‘pytorch_model.bin’

pytorch_model.bin   100%[===================>] 674.12M  15.8MB/s    in 44s     

2022-03-29 23:10:03 (15.3 MB/s) - ‘pytorch_model.bin’ saved [706871064/706871064]



In [ ]:
## We are defining all the needed functions here. 
def inference(data, model, tokenizer):
    # Calculate bleu
    eval_sampler = SequentialSampler(data)
    eval_dataloader = DataLoader(data, sampler=eval_sampler, batch_size=len(data))

    model.eval()
    p = []
    for batch in eval_dataloader:
        batch = tuple(t.to('cpu') for t in batch)
        source_ids, source_mask = batch
        with torch.no_grad():
            preds = model(source_ids=source_ids, source_mask=source_mask)
            for pred in preds:
                t = pred[0].cpu().numpy()
                t = list(t)
                if 0 in t:
                    t = t[: t.index(0)]
                text = tokenizer.decode(t, clean_up_tokenization_spaces=False)
                p.append(text)
    return (p, source_ids.shape[-1])


def get_features(examples, tokenizer):
    features = convert_examples_to_features(
        examples, tokenizer, stage="test"
    )
    all_source_ids = torch.tensor(
        [f.source_ids[: 256] for f in features], dtype=torch.long
    )
    all_source_mask = torch.tensor(
        [f.source_mask[: 256] for f in features], dtype=torch.long
    )
    return TensorDataset(all_source_ids, all_source_mask)


def build_model(model_class, config, tokenizer):
    encoder = model_class(config=config)
    decoder_layer = nn.TransformerDecoderLayer(
        d_model=config.hidden_size, nhead=config.num_attention_heads
    )
    decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
    model = Seq2Seq(
        encoder=encoder,
        decoder=decoder,
        config=config,
        beam_size=10,
        max_length=128,
        sos_id=tokenizer.cls_token_id,
        eos_id=tokenizer.sep_token_id,
    )

    model.load_state_dict(
        torch.load(
            "pytorch_model.bin",
            map_location=torch.device("cpu"),
        ),
        strict=False,
    )
    return model

Now that we have all the needed functions, let's load the baseline model from Hugging Face model hub

In [ ]:
config = RobertaConfig.from_pretrained("microsoft/codebert-base")
tokenizer = RobertaTokenizer.from_pretrained(
    "microsoft/codebert-base", do_lower_case=False
)

model = build_model(
    model_class=RobertaModel, config=config, tokenizer=tokenizer
).to('cpu')

Everything is ready to make predictions!! Let's do it.

In [ ]:
example = [Example(source="def add_tensors(t, t1) -> Any:\n    return t + t1", target=None)]
message, length = inference(get_features(example, tokenizer), model, tokenizer)
print(message)

/content/model.py:219: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


['Add two tensors .']


**AMAZING!!**

We need to be able to run it on a bunch of files and extract the functions from it and then predict their docstrings. How shall we do it?


Codist [tree-hugger](https://github.com/autosoft-dev/tree-hugger) to the rescue!

For the ease of the tutorial we have created a small github example repo with a collection of files. Some of it is coming from Open Source repos and some we created as example files. 

Let's clone that

In [ ]:
!git clone https://github.com/autosoft-dev/example-files.git

Cloning into 'example-files'...
remote: Enumerating objects: 133, done.
remote: Total 133 (delta 0), reused 0 (delta 0), pack-reused 133
Receiving objects: 100% (133/133), 356.91 KiB | 2.23 MiB/s, done.
Resolving deltas: 100% (26/26), done.


We are going to declare a small function that will help us go over each files in a nested directory tree (like the one above we cloned) and get each file at a time. 

In [ ]:
from pathlib import Path

def check_out_path(target_path: Path):
    """"
    This function recursively yields all contents of a pathlib.Path object
    """
    yield target_path
    for file in target_path.iterdir():
        if file.is_dir():
            yield from check_out_path(file)
        else:
            yield file.absolute()


def is_python_file(file_path: Path):
  """
  This little function will help us to filter the result and keep only the python files
  """
  return file_path.is_file() and file_path.suffix == ".py"

In [ ]:
for file_path in check_out_path(Path("example-files")):
  if is_python_file(file_path):
    print(file_path)

/content/example-files/ml_files/ChatterBot/adapters.py
/content/example-files/ml_files/ChatterBot/constants.py
/content/example-files/ml_files/ChatterBot/chatterbot.py
/content/example-files/ml_files/ChatterBot/conversation.py
/content/example-files/ml_files/ChatterBot/corpus.py
/content/example-files/ml_files/ChatterBot/comparisons.py
/content/example-files/ml_files/ChatterBot/ext/django_chatterbot/apps.py
/content/example-files/ml_files/ChatterBot/ext/django_chatterbot/migrations/0007_response_created_at.py
/content/example-files/ml_files/ChatterBot/ext/django_chatterbot/migrations/0004_rename_in_response_to.py
/content/example-files/ml_files/ChatterBot/ext/django_chatterbot/migrations/0011_blank_extra_data.py
/content/example-files/ml_files/ChatterBot/ext/django_chatterbot/migrations/0005_statement_created_at.py
/content/example-files/ml_files/ChatterBot/ext/django_chatterbot/migrations/0012_statement_created_at.py
/content/example-files/ml_files/ChatterBot/ext/django_chatterbot/mig

We are now ready to use tree-hugger to parse all the needed files and let's do that

In [ ]:
# We first create our PythonParser object
from tree_hugger.core import PythonParser

In [ ]:
pp = PythonParser(library_loc="/content/my-languages.so")

In [ ]:
# Let's use the function we defined before to go over all the files.
for file_path in check_out_path(Path("example-files")):
  if is_python_file(file_path):
    # we use one line, super convinient tree-hugger API call to get the needed data
    if pp.parse_file(str(file_path)):
      temp_cache = []
      # The following call returns a dict where each key is a name of a function
      # And each value is a tuple, (function_body, function_docstring)
      func_and_docstr = pp.get_all_function_bodies(strip_docstr=True)
      for func_name, (body, docstr) in func_and_docstr.items():
        example = [Example(source=body, target=None)]
        message, length = inference(get_features(example, tokenizer), model, tokenizer)
        print(func_name, " ".join(message))
      # Let's add the result to the final output

/content/model.py:219: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


get_file_path Get the file path from the given dotted path .
read_corpus Read a corpus .
list_corpus_files Return a list of files in a dotted path .
load_corpus Yields a list of conversations .
model_fn_builder Build a model_fn for TPUEstimator .
model_fn Train the BERT model .
tpu_scaffold Get a tf . scaffold . scaffold .
metric_fn Compute metric function .
get_masked_lm_output Create a masked Lm output layer .
get_next_sentence_output Get the next sentence output .
gather_indexes Gather indices from a tensor .
input_fn_builder Builds the input function .
input_fn Define input function .
_decode_record Decodes a record into int32 .
main Main entry point .
convert_single_example Convert single example to InputFeatures .
file_based_convert_examples_to_features Converts a list of examples into features .
create_int_feature Create a tf . Feature .
file_based_input_fn_builder Build a TensorFlow input function .
_decode_record Decode a record to int32 .
input_fn Input function .
_truncate_s

With this code, you can very easily create a dataset out of your own code files and then test the baseline models against it. 


That was easy!

 As said earlier, codist just relased `docly` we use similar parsing and modelling methods to generate docstring (with arguments and several other things that is not present in this baseline model). If you are interested, please have a look [here](https://codist-ai.com/)

In [ ]:
# Let's use the function we defined before to go over all the files.
counter = 0
for file_path in check_out_path(Path("example-files")):
  if counter > 30:
    break
  if is_python_file(file_path):
    # we use one line, super convinient tree-hugger API call to get the needed data
    if pp.parse_file(str(file_path)):
      counter += 1
      temp_cache = []
      # The following call returns a dict where each key is a name of a function
      # And each value is a tuple, (function_body, function_docstring)
      func_and_docstr = pp.get_all_function_bodies(strip_docstr=True)
      for func_name, (body, docstr) in func_and_docstr.items():
        example = [Example(source=body, target=None)]
        message, length = inference(get_features(example, tokenizer), model, tokenizer)
        if docstr != "":
          print(func_name,
                "\nSummary: [" + " ".join(message) + "]",
                "\nDoc string: [" + docstr + "]")
      # Let's add the result to the final output

/content/model.py:219: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


get_file_path 
Summary: [Get the file path from the given dotted path .] 
Doc string: ["""
    Reads a dotted file path and returns the file path.
    """]
read_corpus 
Summary: [Read a corpus .] 
Doc string: ["""
    Read and return the data from a corpus json file.
    """]
list_corpus_files 
Summary: [Return a list of files in a dotted path .] 
Doc string: ["""
    Return a list of file paths to each data file in the specified corpus.
    """]
load_corpus 
Summary: [Yields a list of conversations .] 
Doc string: ["""
    Return the data contained within a specified corpus.
    """]
model_fn_builder 
Summary: [Build a model_fn for TPUEstimator .] 
Doc string: ["""Returns `model_fn` closure for TPUEstimator."""]
model_fn 
Summary: [Train the BERT model .] 
Doc string: ["""The `model_fn` for TPUEstimator."""]
metric_fn 
Summary: [Compute metric function .] 
Doc string: ["""Computes the loss and accuracy of the model."""]
get_masked_lm_output 
Summary: [Create a masked Lm output layer .